# ex-03 Compare Spatial Indexing Methods - STRTree

When it comes to query spatial data at scale, there’s no concept more useful and important than a spatial index. Spatial indices are a family of algorithms that arrange geometric data for efficient search. For example, check if a point within a polygon(s).

Within the ecosystem of geopandas, there are two spatial indexing methods available:
- [R-Tree](https://en.wikipedia.org/wiki/R-tree)
> R-trees are tree data structures used for spatial access methods, i.e., for indexing multi-dimensional information such as geographical coordinates, rectangles or polygons. 

- [STRTree](https://pygeos.readthedocs.io/en/latest/strtree.html)
> A query-only R-tree created using the Sort-Tile-Recursive (STR) algorithm. For two-dimensional spatial data. The STR packed R-tree is simple to implement and maximizes space utilization; that is, as many leaves as possible are filled to capacity. 

We will compare these two methods in this and next tutorials. The trees look like as the following image.
<img  src='img/rtree.png'>
Image is downloaded from https://github.com/tidwall/rtree/blob/master/cities.png

## 1 Import libs

In [5]:
import itertools
import numpy as np
import pandas as pd 
import geopandas as gpd
from shapely.geometry import Point

import matplotlib.pyplot as plt
%matplotlib inline

## 2 Prepare data

- Read global map

In [2]:
df_new = gpd.read_parquet('data/gadm404_Level1.parquet')

- Make some fake points (1000 points)

In [3]:
s = gpd.GeoSeries(gpd.points_from_xy([-149.955038, -155.623983, 28.174123, 
                                      -53.701103, 10.800129,  116.372710,
                                      174.783448, 175.27606, 147.352620,
                                      -176.474764
                                     ], 
                                     [21.777515, 19.619468,  -32.321729, 
                                      -10.711625,  62.435156, 39.966970,
                                      -36.889234, -37.768188, -42.016550,
                                      -44.021665,
                                     ]))

a = [s] * 100
ss = list(itertools.chain(*a))
ss = gpd.GeoSeries(ss)
len(ss)

1000

## 3 Points in Polygons (PIPs)

- Check Spatial Indexing Method

In [4]:
ssindex = df_new.sindex
ssindex

Once the operationof df_new.sindex finished, the spatial index was created. Now current index is ***STRTree***, which is from PyGEOSSTRTreeIndex. Therefore, the package of pygeos is installed on your computer, the default index is ***STRTree***.

#### Check the speed of PIPs

In [6]:
%%time
idx = ssindex.query_bulk(ss, predicate="within").T

CPU times: total: 1min 29s
Wall time: 1min 29s


The query results only list the points over land.

In [8]:
land_points = ss.iloc[idx[:, 0]]
lons = [ew_point.coords.xy[0][0] for ipt, ew_point in enumerate(land_points)]
lats = [ew_point.coords.xy[1][0] for ipt, ew_point in enumerate(land_points)]

df_land_points = df_new[['State', 'Country']].iloc[idx[:, 1]]
df_land_points['latitude']  = lats
df_land_points['longitude'] = lons

df_land_points.index = idx[:, 0]
df_land_points

,State,Country,latitude,longitude
1,Hawaii,United States,19.619468,-155.623983
2,Eastern Cape,South Africa,-32.321729,28.174123
3,Mato Grosso,Brazil,-10.711625,-53.701103
4,Hedmark,Norway,62.435156,10.800129
5,Beijing,China,39.966970,116.372710
...,...,...,...,...
995,Beijing,China,39.966970,116.372710
996,Auckland,New Zealand,-36.889234,174.783448
997,Waikato,New Zealand,-37.768188,175.276060
998,Tasmania,Australia,-42.016550,147.352620
